# External Monitoring

Say you have a complicated agentic workflow or are running in a restricted environment—whatever the reason, you may not be able to just lift an application or workflow and bring it onto DataRobot to run. For these scenarios, you can still have "bolt-on" monitoring and observability by using DataRobot's agent libraries.

The decorator in `monitoring.py` allows you to monitor any agentic or generative workflow directly using DataRobot's external monitoring.

**11.1 Updates**
Note that in 11.1, we will also be supporting [OpenTelemetry](https://opentelemetry.io/) for monitoring, helping make this process even easier.

## Set-up

The first step is registering an "External Model" with DataRobot and creating a deployment for it. This is giving DataRobot the metadata it needs to start collecting your telemetry data. We will demonstrate this by building a chatbot that helps us discuss sonnets from William Shakespeare.

In [1]:
import datarobot as dr
from datarobot.models import RegisteredModelVersion
import pandas as pd
from dataclasses import dataclass

dr_client = dr.Client()
NAME = "Shakespeare External Model"
INPUT_PROMPT_NAME = "promptText"

In [2]:
# Obtain the Shakespear Base Data

import requests as req

sonnets = req.get(
    "https://raw.githubusercontent.com/enerrio/Generate-Shakespeare-Sonnets/refs/heads/master/sonnets.txt"
).text.split("\n\n")

dataset = dr.Dataset.create_from_in_memory_data(
    pd.DataFrame([(s, s) for s in sonnets], columns=[INPUT_PROMPT_NAME, "output"]),
    fname="Shakespeare Sonnets Dataset",
)

In [5]:
@dataclass
class ExternalModelInfo:
    registered_model_id: str
    registered_model_version_id: str


reg_models = [m for m in dr.RegisteredModel.list() if m.name == NAME]
if len(reg_models) == 0:
    data = {
        "name": NAME,
        "modelDescription": {"buildEnvironmentType": "Other", "modelName": NAME},
        "textGeneration": {"prompt": INPUT_PROMPT_NAME},
        "registeredModelName": NAME,
        "target": {"name": "output", "type": "TextGeneration"},
        "datasets": {"trainingDataCatalogId": dataset.id},
    }
    resp = dr_client.post("modelPackages/fromJSON", json=data)
    resp = resp.json()
    external_model = ExternalModelInfo(
        registered_model_id=resp["registeredModelId"],
        registered_model_version_id=resp["id"],
    )

else:
    data = {
        "name": NAME,
        "modelDescription": {"buildEnvironmentType": "Other", "modelName": NAME},
        "registeredModelId": reg_models[0].id,
        "textGeneration": {"prompt": INPUT_PROMPT_NAME},
        "target": {"name": "output", "type": "TextGeneration"},
        "datasets": {"trainingDataCatalogId": dataset.id},
    }

    resp = dr_client.post("modelPackages/fromJSON", json=data)
    resp = resp.json()
    external_model = ExternalModelInfo(
        registered_model_id=resp["registeredModelId"],
        registered_model_version_id=resp["id"],
    )

Now we also have to create metadata for where this model runs and deploy it. 

In [8]:
pred_environment = dr.PredictionEnvironment.create(
    name="Shakespeare External Model Environment",
    platform=dr.PredictionEnvironmentPlatform.OTHER,
)

deployment = dr.Deployment.create_from_registered_model_version(
    external_model.registered_model_version_id,
    label="Shakespeare External Model Deployment",
    prediction_environment_id=pred_environment.id,
)

deployment.update_association_id_settings(
    column_names=["id"], required_in_prediction_requests=False
)
deployment.update_predictions_data_collection_settings(enabled=True)
deployment.update_drift_tracking_settings(
    target_drift_enabled=False, feature_drift_enabled=True
)

## Configure OTEL

In addition to logging the prompt and response, we can log tracing and telemetry via the OTEL framework. OTEL is set-up by activating key environment variables and ensuring that the telemetry packages are set up and installed. Depending on your workflow, you may need to add more telemetry insturmentation packages and activate them. 

In [9]:
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.trace import set_tracer_provider
from opentelemetry.sdk.resources import Resource
from opentelemetry import trace
from opentelemetry.instrumentation.httpx import HTTPXClientInstrumentor

HTTPXClientInstrumentor().instrument()

from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

OpenAIInstrumentor().instrument()


import os


def configure_tracer(
    api_key: str, endpoint_without_api: str = "https://app.datarobot.com"
):
    entity_type = "deployment"
    entity_id = deployment.id

    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = (
        f"X-DataRobot-Entity-Id={entity_type}-{entity_id},X-DataRobot-Api-Key={api_key}"
    )
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://app.datarobot.com/otel"

    resource = Resource.create()
    otlp_exporter = OTLPSpanExporter()
    provider = TracerProvider(resource=resource)
    provider.add_span_processor(BatchSpanProcessor(otlp_exporter))
    trace.set_tracer_provider(provider)


configure_tracer(dr_client.token)

Now let's create our analysis app. 

In [10]:
from openai import OpenAI
from random import choice

oai_client = OpenAI(
    api_key=dr_client.token, base_url=dr_client.endpoint + "/genai/llmgw"
)

import monitoring
from importlib import reload

reload(monitoring)


@monitoring.dr_monitor(
    deployment_id=deployment.id,
    model_id=deployment.model["id"],
    datarobot_api_token=dr_client.token,
    datarobot_endpoint=dr_client.endpoint,
)
async def describe_sonnet(sonnet=choice(sonnets)):
    response = oai_client.chat.completions.create(
        model="azure/gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """You are a helpful assistant. 
             Pretend you are an expert in Shakespeare's works.""",
            },
            {"role": "user", "content": f"Describe the following sonnet:\n\n{sonnet}"},
        ],
    )
    resp = response.choices[0].message.content.strip()
    print(resp)
    return resp


r = await describe_sonnet()

This sonnet is Sonnet 29 from William Shakespeare's collection. It belongs to the Fair Youth sequence, a series of 126 sonnets written to a young man of great beauty and promise.

In this sonnet, Shakespeare explores the theme of true worth and value, contrasting societal measures of success—such as birth, wealth, physical prowess, and the status derived from possessions—with the profound and personal value of love. 

The poem opens with the speaker acknowledging that people tend to take pride in various aspects of their lives: their lineage, skills, material wealth, physical strength, and even the status associated with their clothing and animals. Each of these elements brings its own joy and satisfaction, indicating that people find value in different things. However, the speaker asserts that these distinctions do not define him or his own measures of value.

Importantly, the speaker emphasizes that his attachment to love surpasses all these societal measures. The beloved's love hold

In [11]:
number_of_requests = deployment.get_service_stats()

number_of_requests

ServiceStats(690e58e15106399b945b3436 | 2025-10-31 21:00:00+00:00 - 2025-11-07 21:00:00+00:00)

### Cleanup

In [ ]:
deployment.delete()
pred_environment.delete()
dr_client.delete(f"registeredModels/{external_model.registered_model_id}")